# match function
- [직접 구현하지 않음](/2_objectdetection/utils/match.py) 
- 정답(BBox)과 가까운 DBox 추출: Jaccard(IOU) 사용해 BBox, DBox 간 유사도 측정
- Negative DBox(정답 데이터가 없는 DBox)
    > $IOU< 0.5$ 
    >
    > background class도 index 0으로 취급하여 학습에 사용
- Positive DBox: 해당하는 정답 BBox의 object class를 해당 DBox 예측 결과의 정답 class로 둠(지도 데이터)
    > $IOU\geq 0.5$ 
- DBox의 좌표 정보(loc), DBox가 감지되는 클래스를 따로 고려함
    > jaccard 계수는 loc만 고려
    > - 신뢰도(conf)가 높은 class 등은 고려하지 않음
    > 
    > 미리 준비한 DBox와 정답 BBox에 대해서만 jaccard 계산
    > - 추정 BBox와 정답 BBox의 jaccard 계수 처리하지 않음


# hard negative mining
- Negative DBox 중 학습에 사용될 DBox 추출
    > Negative DBox를 전부 학습에 사용하면 background class 학습 횟수가 실제 object class 비해 매우 커지기 때문에 균형을 위한 조치
    > 
    > loss가 높은(label 예측이 잘 되지 않은) 것을 우선적으로 선택
    > - background class를 background class라고 예측하지 못한 DBox를 Negative DBox라 하고 이를 우선적으로 학습
- Negative DBox:Positive DBox=n:1 로 조정
    > n(hyperparameter): neg_pos


# loss function 

### Smooth L1 Loss 
- Positive DBox offset 예측 loc: DBox, 정답 BBox가 되기 위한 보정 값을 예측하는 regression
- 물체를 발견한 DBox(Positive DBox)의 offset만 계산하도록 함 
    > Negative DBox label이 0(background)로 되어있어 BBox가 존재하지 않아 offset 또한 없음

### cross entropy
- object class label 예측

### MultiBoxLoss class

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from utils.match import match

In [3]:


class MultiBoxLoss(nn.Module):
    def __init__(self,jaccard_thresh=0.5, neg_pos=3,device='cpu'):
        super(MultiBoxLoss,self).__init__()
        self.jaccard_thresh=jaccard_thresh #for match function
        self.negpos_ratio=neg_pos #negative dbox:positive dbox
        self.device=device

    def forward(self,predictions,targets):
        loc_data,conf_data,dbox_list=predictions
        num_batch=loc_data.size(0)
        num_dbox=loc_data.size(1)
        num_classes=conf_data.size(2)

        '''dbox에 가장 가까운 정답 bbox의 정보 저장''' 

        #init
        conf_t_label=torch.LongTensor(num_batch,num_dbox).to(self.device) #label
        loc_t=torch.Tensor(num_batch,num_dbox,4).to(self.device) #location 

        for idx in range(num_batch):
            # 현재 minibatch의 정답 annotation 
            bbox=targets[idx][:,:-1].to(self.device) #bbox 
            bbox_labels=targets[idx][:,-1].to(self.device) #label: [object_1, label_1, o_2,l_2,...]

            # init
            dbox=dbox_list.to(self.device)

            # get label, location info using match function
            # if jaccard overlap< 0.5, set label index 0(background class)
            
            variance=[0.1,0.2] #DBox에서 BBox로의 보정 과정에서 필요한 계수
            match(self.jaccard_thresh,bbox,dbox,variance,bbox_labels,loc_t,conf_t_label,idx)
        
        pos_mask=conf_t_label>0 #object 감지된 bbox 추출 (positive dbox가 계산된 bbox)

        '''location loss: Smooth L1 Loss'''
        pos_idx=pos_mask.unsqueeze(pos_mask.dim()).expand_as(loc_data) #pos_mask를 loc_data 크기로 변형
        loc_p=loc_data[pos_idx].view(-1,4) #dbox location info 
        loc_t=loc_t[pos_idx].view(-1,4) #bbox location info
        
        loss_l=F.smooth_l1_loss(loc_p,loc_t,reduction='sum')

        '''class loss: cross entropy'''
        # 1. CLASS PREDICTION LOSS
        batch_conf=conf_data.view(-1,num_classes)
        loss_c=F.cross_entropy(batch_conf,conf_t_label.view(-1),reduction='none') # 차원 보존을 위해 더하지 않고 reduction='none'


        # 2. HARD NEGATIVE MINING

        # (1) if object detect(label > 1), loss will be 0
        num_pos=pos_mask.long().sum(1,keepdim=True) #object class의 예측 수 for each minibatch
        loss_c=loss_c.view(num_batch,-1)
        loss_c[pos_mask]=0 


        # (2) background dbox 개수 조정
        num_neg=torch.clamp(num_pos*self.negpos_ratio,max=num_dbox) # object class(num_pos): background class = self.negpos_ratio:1


        # (3) loss 낮은 dbox 제외

        # loss 크기 큰 순서대로 정렬한 idx 기억
            # loss_idx[idx_rank[0]]=원래 loss_c의 0번째 요소 
        _,loss_idx=loss_c.sort(1,descending=True) #loss 큰 순으로 정렬
            # loss_idx: 내림차순으로 정렬된 loss의 원래 idx 값
        _,idx_rank=loss_idx.sort(1) #rank 지정
            # idx_rank: 다시 원 상태로 되돌린 후, loss_idx의 순서 기억 -> 내림차순 시 새로운 idx
        
        #num_neg보다 loss 큰(idx_rank 값 낮은) dbox 추출
        neg_mask=idx_rank<(num_neg).expand_as(idx_rank) 


        # (4) hard negative mining
        
        pos_idx_mask=pos_mask.unsqueeze(2).expand_as(conf_data) #positive dbox conf
        neg_idx_mask=neg_mask.unsqueeze(2).expand_as(conf_data) #negative dbox conf

        # pos나 neg 중 mask 가 1인 index를 추출 
        conf_hnm=conf_data[(pos_idx_mask+neg_idx_mask).gt(0)].view(-1,num_classes) 
        conf_t_label_hnm=conf_t_label[(pos_mask+neg_mask).gt(0)] 
        
        
        # (5) loss 
        loss_c=F.cross_entropy(conf_hnm,conf_t_label_hnm,reduction='sum')

        N=num_pos.sum() #object detect된 bbox 개수

        loss_l/=N
        loss_c/=N

        return loss_l,loss_c